In [ ]:
pip install --upgrade gensim

In [ ]:
import gensim.downloader
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
w2v = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 781.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c76bd7350ad47d91e9850761f9e1b15ed475e44f801e2244c81253209d11820d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
pip install nltk

In [ ]:
import os
from tensorflow import keras
import pandas as pd
import numpy as np
from seqeval.metrics import f1_score, classification_report

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import TimeDistributed

**Question 1.1**  
Based on word2vec embeddings you have downloaded, use cosine similarity to find the most similar
word to each of these words: (a) “student”; (b) “Apple”; (c) “apple”. Report the most similar word
and its cosine similarity.


In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models
# https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.most_similar.html
print("(a):", w2v.most_similar('student')[0])
print("(b):", w2v.most_similar('Apple')[0])
print("(c):", w2v.most_similar('apple')[0])

(a): ('students', 0.7294867038726807)
(b): ('Apple_AAPL', 0.7456986308097839)
(c): ('apples', 0.720359742641449)


**Question 1.2**  
(a) Describe the size (number of sentences) of the training, development and test file for CoNLL2003.
Specify the complete set of all possible word labels based on the tagging scheme (IO, BIO,
etc.) you chose.

In [ ]:
testaPath = keras.utils.get_file(
    fname="eng.testa",
    origin="https://github.com/TheAnig/NER-LSTM-CNN-Pytorch/raw/master/data/eng.testa",
    extract=True,
)
testbPath =  keras.utils.get_file(
    fname="eng.testb",
    origin="https://github.com/TheAnig/NER-LSTM-CNN-Pytorch/raw/master/data/eng.testb",
    extract=True,
)
trainPath =  keras.utils.get_file(
    fname="eng.train",
    origin="https://github.com/TheAnig/NER-LSTM-CNN-Pytorch/raw/master/data/eng.train",
    extract=True,
)

In [ ]:
def getData(path):
  with open(path, 'r') as file:
    dataset = file.read()
  lines = dataset.split('\n')
  split_lines = [line.split() for line in lines]
  max_words = max(len(line) for line in split_lines)
  split_lines = [line + [''] * (max_words - len(line)) for line in split_lines]
  dataset = pd.DataFrame(split_lines)
  dataset.columns = [f'col{i}' for i in range(max_words)]
  dataset = dataset.drop(columns=['col1', 'col2'])
  dataset.rename(columns={'col0': 'word', 'col3': 'label'}, inplace=True)
  return dataset

lastRow = {'word': '', 'label': ''}
lastRow_df = pd.DataFrame([lastRow])

train = getData(trainPath)
development = getData(testaPath)
development = pd.concat([development, lastRow_df], ignore_index=True)
test = getData(testbPath)

category_mapping = {'': 0,'I-LOC': 1, 'B-ORG': 2, 'I-ORG': 3, 'I-PER': 4, 'B-LOC': 5, 'I-MISC': 6, 'B-MISC': 7, 'O': 8}
train["encoded_label"] = train["label"].map(category_mapping)
development["encoded_label"] = development["label"].map(category_mapping)
test["encoded_label"] = test["label"].map(category_mapping)

In [ ]:
print(train.tail())
print(development.tail())
print(test.tail())

              word  label  encoded_label
219549     Lincoln  I-ORG              3
219550           2      O              8
219551                                 0
219552  -DOCSTART-      O              8
219553                                 0
               word  label  encoded_label
55039      Newsroom  I-ORG              3
55040  880-2-506363      O              8
55041                                   0
55042    -DOCSTART-      O              8
55043                                   0
             word  label  encoded_label
50345       Bobby  I-PER              4
50346           .      O              8
50347                                 0
50348  -DOCSTART-      O              8
50349                                 0


In [ ]:
def countSize(df):
  return (df.applymap(lambda x: x == '').any(axis=1)).sum()

def getCompleteLabelSet(train, development, test):
    label_set = set(train["label"].unique()) | set(development["label"].unique()) | set(test["label"].unique())
    return list(label_set)

print("Size of training file: ", countSize(train))
print("Size of development file: ", countSize(development))
print("Size of test file: ", countSize(test))

print("Complete Set of all word labels (IOB1):", getCompleteLabelSet(train, development, test))

Size of training file:  14987
Size of development file:  3466
Size of test file:  3684
Complete Set of all word labels (IOB1): ['', 'B-ORG', 'I-PER', 'I-MISC', 'I-LOC', 'O', 'I-ORG', 'B-LOC', 'B-MISC']


(b) Choose an example sentence from the training set of CoNLL2003 that has at least two named
entities with more than one word. Explain how to form complete named entities from the label
for each word, and list all the named entities in this sentence.


In [ ]:
def getNamedEntitiesForEachSentence(df, namedEntitylabels = ['I-LOC', 'B-ORG', 'I-ORG', 'I-PER', 'B-LOC', 'I-MISC', 'B-MISC']):
  targetSentences = []
  sentenceArray = []
  NECount = 0
  MWNECount = 0
  NEWordCount = 0
  NEArray = []
  NE = ""
  previousNELabel = None

  for index,row in df.iterrows():
    # if word is a period, it symbolises the end of a sentence
    if row["word"] == "":
      # evaluate the previous named entity if present
      if NEWordCount > 1:
        MWNECount += 1
        NECount += 1
        NEArray.append(NE)

      # save other single word named entities
      elif NEWordCount == 1 :
        NECount += 1
        NEArray.append(NE)
      sentenceArray.append([row["word"], row["label"]])

      # for each Sentence, check if the number of Multi-Word Named Entities is >= 2
      if NECount >= 2:
        # if yes, append to targetSentences
        targetSentences.append([sentenceArray,NEArray,NECount, MWNECount])
      # set all variables to process the next sentence
      sentenceArray = []
      NECount = 0
      MWNECount = 0
      NEWordCount = 0
      NEArray = []
      NE = ""
      previousLabel = None
      continue

    # if word belongs has a named entity label
    elif row["label"] in namedEntitylabels:
      # get the label type and the position
      position, label = row["label"].split("-")
      # if a word with the same label has appeared before
      # and the current word is not beginning word
      # the previous and current words can be grouped togther as 1 named entity
      if previousNELabel is not None and previousNELabel == label and position != "B":
        NEWordCount += 1
        NE += row["word"] + " "

      else:
        # if the current word is the start of a new named entity
        # evaluate the previous named entity
        if NEWordCount > 1:
          NECount += 1
          MWNECount += 1
          NEArray.append(NE)

        # save other single word named entities
        elif NEWordCount == 1 :
          NECount += 1
          NEArray.append(NE)

        #and track the current new named entity
        previousNELabel = label
        NEWordCount = 1
        NE = ""
        NE += row["word"]+" "

    # if word does not have a named entity label
    else:
      # if there was a named enitity before current word,
      # evaluate it
      if NEWordCount > 1:
        MWNECount += 1
        NECount += 1
        NEArray.append(NE)
      elif NEWordCount == 1:
        NECount += 1
        NEArray.append(NE)

      #reset values
      NE = ""
      NEWordCount = 0

    sentenceArray.append([row["word"],row["label"]])

  return targetSentences

def getSentenceFromRowArray(array):
  sentence = ""
  for word, label in array:
    sentence += word
    sentence += " "
  return sentence

In [ ]:
sentences = getNamedEntitiesForEachSentence(train)
sentences = sorted(sentences, key=lambda x: x[3])

In [ ]:
len(sentences)

6136

In [ ]:
# a = []
# for i in range(0,len(sentences)):
#   for j in sentences[i][0]:
#     if j[1].split("-")[0] == "B" and sentences[i][3] > 0:
#       a.append(i)
# a = list(set(a))

# for i in a:
#   print(i)
#   print("Example Sentence:", getSentenceFromRowArray(sentences[i][0]))
#   print("All named entities:",sentences[i][1])

In [ ]:
random = 6041
print("Example Sentence:", getSentenceFromRowArray(sentences[random][0]))
print("All named entities:",sentences[random][1])

print("\nConsecutive entity labels with the same class (e.g PER) can be grouped together to get a named entity. B tags in the label can be used to split a chunk of entity words into meaningful groups.\n")

print("Labels (for reference):",sentences[random][0], "\n")

Example Sentence: Sosa , a leading candidate for National League Most Valuable Player honours , was injured August 20th when he was hit by a Mark Hutton pitch in the first inning of an 8-1 victory over the Florida Marlins .  
All named entities: ['Sosa ', 'National League ', 'Most Valuable Player ', 'Mark Hutton ', 'Florida Marlins ']

Consecutive entity labels with the same class (e.g PER) can be grouped together to get a named entity. B tags in the label can be used to split a chunk of entity words into meaningful groups.

Labels (for reference): [['Sosa', 'I-PER'], [',', 'O'], ['a', 'O'], ['leading', 'O'], ['candidate', 'O'], ['for', 'O'], ['National', 'I-MISC'], ['League', 'I-MISC'], ['Most', 'B-MISC'], ['Valuable', 'I-MISC'], ['Player', 'I-MISC'], ['honours', 'O'], [',', 'O'], ['was', 'O'], ['injured', 'O'], ['August', 'O'], ['20th', 'O'], ['when', 'O'], ['he', 'O'], ['was', 'O'], ['hit', 'O'], ['by', 'O'], ['a', 'O'], ['Mark', 'I-PER'], ['Hutton', 'I-PER'], ['pitch', 'O'], ['in',

**Question 1.3**  
(a) Discuss how you deal with new words in the training set which are not found in the pretrained
dictionary. Likewise, how do you deal with new words in the test set which are not found in
either the pretrained dictionary or the training set? Show the corresponding code snippet.



In [ ]:
def getSentenceAndVocab(df):
  allSentences = []
  sentenceArray = []
  newVocab = {}
  keyValue = 0

  for index,row in df.iterrows():
    if row["word"] == "":
      sentenceArray.append([row["word"], row["encoded_label"]])
      allSentences.append(sentenceArray)
      sentenceArray = []
      continue
    else:
      try:
        w2v[row["word"]]
      except:
          try:
              newVocab[row["word"]]
          except:
              newVocab[row["word"]] = keyValue
              keyValue += 1

    sentenceArray.append([row["word"],row["encoded_label"]])

  return allSentences, newVocab

In [ ]:
trainSentences, trainVocab = getSentenceAndVocab(train)
developmentSentences, developmentVocab = getSentenceAndVocab(development)
testSentences, testVocab = getSentenceAndVocab(test)

In [ ]:
from nltk.util import ngrams
import numpy as np

def getNgrams(train):
  ngramsDict = {}
  for idx, row in train.iterrows():
    try:
      w2v[row["word"]]
    except:
      continue
    if row["word"] not in ngramsDict:
      ngramsDict[row["word"]] = list(ngrams(row["word"], 3)) + list(ngrams(row["word"], 2))
  return ngramsDict

def getMostSimilarKeys(OOVWord, ngramsDict):
  OOVNgrams = list(ngrams(OOVWord, 3)) + list(ngrams(OOVWord, 2))
  similarKeys = []
  keyCount = []
  for n in ngramsDict.keys():
    currentCount = len(set(OOVNgrams).intersection(set(ngramsDict[n])))
    keyCount.append([n,currentCount, -len(ngramsDict[n])])

  keyCount = sorted(keyCount, key = lambda x: (x[1], x[2]))

  for value in keyCount[-3:]:
    similarKeys.append(value[0])
  return similarKeys

def getOOVVector(similarKeys, OOVWord, vectorDim = 300):
  random_vectors = []

  np.random.seed(seed = 42)
  random_vectors.append((np.random.rand(vectorDim) * 2.0) - 1.0)

  random_vector = np.mean(random_vectors, axis=0)
  random_vector = random_vector / np.linalg.norm(random_vector)

  sum_vector = 0
  for word in similarKeys:
    sum_vector += w2v[word]
  mean_vector =  np.divide(sum_vector, len(similarKeys))
  final_vector = (random_vector * 0.3 + mean_vector * 0.7)
  final_vector = final_vector / np.linalg.norm(final_vector)

  return final_vector

def embedNewWordsNgrams(trainVocab, ngramsDict):
  for key, value in trainVocab.items():
    similarKeys = getMostSimilarKeys(key, ngramsDict)
    trainVocab[key] = getOOVVector(similarKeys, key)


ngramsDictTrain = getNgrams(train)
embedNewWordsNgrams(trainVocab, ngramsDictTrain)

ngramsDictVal = getNgrams(development)
embedNewWordsNgrams(developmentVocab, ngramsDictVal)

ngramsDictTest = getNgrams(test)
embedNewWordsNgrams(testVocab, ngramsDictTest)

In [ ]:
maxLen = max(max([len(s) for s in trainSentences]),
                  max([len(s) for s in developmentSentences]),
                  max([len(s) for s in testSentences]))
print(maxLen)

125


In [ ]:
# 300 is the size of vector embedding
xTrainArray = np.zeros((len(trainSentences), maxLen, 300))
xDevelopmentArray = np.zeros((len(developmentSentences), maxLen, 300))
xTestArray = np.zeros((len(testSentences), maxLen, 300))
yTrainArray = np.zeros((len(trainSentences), maxLen))
yDevelopmentArray = np.zeros((len(developmentSentences), maxLen))
yTestArray = np.zeros((len(testSentences), maxLen))

In [ ]:
def createXArray(sentences, array, vocab):
  for sentence_idx, sentence in enumerate(sentences):
      arrayPos = 0  # Initialize the array position for each sentence
      for word in sentence:
          if arrayPos >= maxLen: # truncate
              break  # If the sentence exceeds the maximum length, exit the loop
          if word[0] == "":
              for i in range(arrayPos, maxLen): # padding
                  array[sentence_idx][i] = 0
          else:
              try: # check whether it is in pretrained word2vec
                  array[sentence_idx][arrayPos] = w2v[word[0]]
              except: # check whether it is in vocab
                  array[sentence_idx][arrayPos] = vocab[word[0]]
          arrayPos += 1

In [ ]:
def createYArray(sentences, array):
    for sentence_idx, sentence in enumerate(sentences):
        arrayPos = 0  # Initialize the array position for each sentence
        for word in sentence:
            if arrayPos >= maxLen:
                break  # If the sentence exceeds the maximum length, exit the loop
            if word[0] == "":
                for i in range(arrayPos, maxLen):
                    array[sentence_idx][i] = 0
            else:
                array[sentence_idx][arrayPos] = word[1]
            arrayPos += 1

In [ ]:
createXArray(trainSentences, xTrainArray, trainVocab)
createYArray(trainSentences, yTrainArray)

createXArray(developmentSentences, xDevelopmentArray, developmentVocab)
createYArray(developmentSentences, yDevelopmentArray)

createXArray(testSentences, xTestArray, testVocab)
createYArray(testSentences, yTestArray)

In [ ]:
input = Input(shape=(maxLen, 300))
model = LSTM(units=300,return_sequences=True)(input)
output = TimeDistributed(Dense(9, activation="softmax"))(model)
model = Model(input,output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 125, 300)]        0         
                                                                 
 lstm (LSTM)                 (None, 125, 300)          721200    
                                                                 
 time_distributed (TimeDist  (None, 125, 9)            2709      
 ributed)                                                        
                                                                 
Total params: 723909 (2.76 MB)
Trainable params: 723909 (2.76 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Do yall want to try bidirectional? Should have better perf. Not sure if it will work
# https://www.kaggle.com/code/rahulkumarpatro/named-entity-recognition-using-lstm
from tensorflow.keras.layers import SpatialDropout1D, Bidirectional
input = Input(shape=(maxLen, 300))
model = SpatialDropout1D(0.2)(input)
model = Bidirectional(LSTM(units=150,return_sequences=True, recurrent_dropout=0.2))(input)
output = TimeDistributed(Dense(9, activation="softmax"))(model)
model = Model(input,output)
model.summary()

(b) Describe what neural network you used to produce the final vector representation of each
word and what are the mathematical functions used for the forward computation (i.e., from
the pretrained word vectors to the final label of each word). Give the detailed setting of the
network including which parameters are being updated, what are their sizes, and what is the
length of the final vector representation of each word to be fed to the softmax classifier.

In [ ]:
units=300
W = model.layers[1].get_weights()[0]
U = model.layers[1].get_weights()[1]
b = model.layers[1].get_weights()[2]

W_i = W[:, :units]
W_f = W[:, units: units * 2]
W_c = W[:, units * 2: units * 3]
W_o = W[:, units * 3:]

U_i = U[:, :units]
U_f = U[:, units: units * 2]
U_c = U[:, units * 2: units * 3]
U_o = U[:, units * 3:]

b_i = b[:units]
b_f = b[units: units * 2]
b_c = b[units * 2: units * 3]
b_o = b[units * 3:]

In [ ]:
print("Weights for the forget gate:")
print("W_f: ", W_f.shape)
print("U_f: ", U_f.shape)
print("b_f: ", b_f.shape)
print("Weights for the input gate:")
print("W_i: ", W_i.shape)
print("U_i: ", U_i.shape)
print("b_i: ", b_i.shape)
print("Weights for the output gate:")
print("W_o: ", W_o.shape)
print("U_o: ", U_o.shape)
print("b_o: ", b_o.shape)
print("Weights for the candidate cell gate:")
print("W_c: ", W_c.shape)
print("U_c: ", U_c.shape)
print("b_c: ", b_c.shape)

Weights for the forget gate:
W_f:  (300, 300)
U_f:  (300, 300)
b_f:  (300,)
Weights for the input gate:
W_i:  (300, 300)
U_i:  (300, 300)
b_i:  (300,)
Weights for the output gate:
W_o:  (300, 300)
U_o:  (300, 300)
b_o:  (300,)
Weights for the candidate cell gate:
W_c:  (300, 300)
U_c:  (300, 300)
b_c:  (300,)


(c) Report how many epochs you used for training, as well as the running time.

Used 3 epochs

(d) Report the f1 score on the test set, as well as the f1 score on the development set for each
epoch during training.


In [ ]:
class F1Callback(tf.keras.callbacks.Callback):

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def __init__(self, x_dev, y_dev, id2label, pad_value=0):
        """
        Args:
            id2label (dict): id to label mapping.
            (e.g. {1: 'B-LOC', 2: 'I-LOC'})
            pad_value (int): padding value.
        """
        super(F1Callback, self).__init__()
        self.id2label = id2label
        self.pad_value = pad_value
        self.x_dev = x_dev
        self.y_dev = y_dev

    def find_pad_index(self, array):
        """Find padding index.
        Args:
            array (list): integer list.
        Returns:
            idx: padding index.
        Examples:
             >>> array = [1, 2, 0]
             >>> self.find_pad_index(array)
             2
        """
        try:
            return list(array).index(self.pad_value)
        except ValueError:
            return len(array)
    def get_length(self, y):
        """Get true length of y.
        Args:
            y (list): padded list.
        Returns:
            lens: true length of y.
        Examples:
            >>> y = [[1, 0, 0], [1, 1, 0], [1, 1, 1]]
            >>> self.get_length(y)
            [1, 2, 3]
        """
        lens = [self.find_pad_index(row) for row in y]
        return lens

    def convert_idx_to_name(self, y, lens):
        """Convert label index to name.
        Args:
            y (list): label index list.
            lens (list): true length of y.
        Returns:
            y: label name list.
        Examples:
            >>> # assumes that id2label = {1: 'B-LOC', 2: 'I-LOC'}
            >>> y = [[1, 0, 0], [1, 2, 0], [1, 1, 1]]
            >>> lens = [1, 2, 3]
            >>> self.convert_idx_to_name(y, lens)
            [['B-LOC'], ['B-LOC', 'I-LOC'], ['B-LOC', 'B-LOC', 'B-LOC']]
        """
        y = [[self.id2label[idx] for idx in row[0:l-1]]
             for row, l in zip(y, lens)]
        return y
    def predict(self, X, y):
        """Predict sequences.
        Args:
            X (list): input data.
            y (list): tags.
        Returns:
            y_true: true sequences.
            y_pred: predicted sequences.
        """
        y_pred = self.model.predict(xDevelopmentArray)

        # reduce dimension.
        y_true = np.argmax(y, -1)
        y_pred = np.argmax(y_pred, -1)

        lens = self.get_length(y_true)

        y_true = self.convert_idx_to_name(y_true, lens)
        y_pred = self.convert_idx_to_name(y_pred, lens)

        return y_true, y_pred
    def on_epoch_end(self, epoch, logs=None):
        # Predict on the development set
        y_pred = self.model.predict(xDevelopmentArray)

        # Convert one-hot encoded labels to class indices
        y_true = self.y_dev
        y_pred = np.argmax(y_pred, axis=-1)

        lens = self.get_length(y_true)
        print(lens)
        y_true = self.convert_idx_to_name(y_true, lens)
        y_pred = self.convert_idx_to_name(y_pred, lens)

        score = f1_score(y_true, y_pred)

        print(' - f1: {:04.2f}'.format(score * 100))
        print(classification_report(y_true, y_pred, digits=4))

        epoch_duration = time.time() - self.start_time
        print(f"Epoch {epoch + 1} took {epoch_duration} seconds")

labels = {0:'Pd',1:'I-LOC', 2:'B-ORG', 3:'I-ORG', 4:'I-PER', 5:'B-LOC', 6:'I-MISC', 7:'B-MISC', 8:'O'}
f1score = F1Callback(x_dev = xDevelopmentArray, y_dev = yDevelopmentArray,id2label= labels )

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(xTrainArray,yTrainArray,batch_size=32,epochs=3,callbacks=[f1score])

In [ ]:
devPreds = model.predict(xDevelopmentArray)
testPreds = model.predict(xTestArray)

In [ ]:
def getTagsForEachSentence(predictedLabels, sentences):
    outputArray = []
    for i in range(0, len(sentences)):
        for j in range(0, len(sentences[i])):
            outputArray.append(predictedLabels[i][j])

    return outputArray

In [ ]:
devLabels = np.argmax(devPreds, axis=-1)
print(classification_report(getTagsForEachSentence(devLabels, developmentSentences), getTagsForEachSentence(yDevelopmentArray, developmentSentences)))

In [ ]:
testLabels = np.argmax(testPreds, axis=-1)
print(classification_report(getTagsForEachSentence(testLabels, testSentences), getTagsForEachSentence(yTestArray, testSentences)))